In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import os
from util import HISTORICAL_FILE_JOIN, RESOURCE_DIR

In [9]:
df = pd.read_excel(HISTORICAL_FILE_JOIN, keep_default_na=False)
#reads projects from Excel file

len(df)
df.head()

,PROJECTS,COMMITS,Memcached,Microsoft Azure Table Storage,FileMaker,Riak KV,Hive,Redis,Aerospike,Google Cloud Datastore,...,Teradata,PostGIS,Etcd,MySQL,Join_Ignite-Sql,Join_Ignite-NoSql,Join_MarkLogic-Sql,Join_MarkLogic-NoSql,Join_Virtuoso-Sql,Join_Virtuoso-NoSql
0,Activiti,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,Activiti,2,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,Activiti,3,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,Activiti,4,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,Activiti,5,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


Removing spaces from column names

In [10]:
#df2.rename(columns={'MS Access': 'MS_Access', 'IBM DB2': 'IBM_DB2','Cosmos BD':'Cosmos_BD','MS SQL Server':'MS_SQL_Server'}, inplace=True)
df.rename(columns={'Join_Ignite-Sql':'Join_Ignite_Sql','Join_Ignite-NoSql':'Join_Ignite_NoSql','Join_MarkLogic-Sql':'Join_MarkLogic_Sql',
                    'Join_MarkLogic-NoSql':'Join_MarkLogic_NoSql','Join_Virtuoso-Sql':'Join_Virtuoso_Sql','Join_Virtuoso-NoSql':'Join_Virtuoso_NoSql'}, inplace=True)
df.rename(columns={'Microsoft Azure Table Storage': 'MicrosoftAzureTableStorage'}, inplace=True)
df.rename(columns={'IBM DB2':'IBM_DB2','Firebase Realtime':'FirebaseRealtime'}, inplace=True)
df.rename(columns={'Influx DB':'Influx_DB', 'Amazon Redshift':'AmazonRedshift','MS SQL Server': 'MS_SQL_Server'}, inplace=True)
df.rename(columns={'Microsoft Azure CosmosDB':'MicrosoftAzureCosmosDB', 'MS Access':'MS_Access','SAP Adaptive Server':'SAPAdaptiveServer'}, inplace=True)
df.rename(columns={'Google Cloud Datastore': 'GoogleCloudDatastore','Riak KV':'Riak_KV', 'Google BigQuery':'GoogleBigQuery'}, inplace=True)
print(df.columns)

Index(['PROJECTS', 'COMMITS', 'Memcached', 'MicrosoftAzureTableStorage',
       'FileMaker', 'Riak_KV', 'Hive', 'Redis', 'Aerospike',
       'GoogleCloudDatastore', 'MariaDB', 'Neo4j', 'H2', 'GoogleBigQuery',
       'HBase', 'Netezza', 'Elasticsearch', 'SapHana', 'Oracle', 'OrientDB',
       'DynamoDB', 'Firebird', 'Informix', 'IBM_DB2', 'Couchbase',
       'FirebaseRealtime', 'MongoDB', 'Hazelcast', 'Impala', 'Influx_DB',
       'SQLite', 'Solr', 'Snowflake', 'AmazonRedshift', 'PostgreSQL', 'Realm',
       'Cassandra', 'MS_SQL_Server', 'Ehcache', 'MicrosoftAzureCosmosDB',
       'MS_Access', 'SAPAdaptiveServer', 'CouchDB', 'ClickHouse', 'Vertica',
       'Teradata', 'PostGIS', 'Etcd', 'MySQL', 'Join_Ignite_Sql',
       'Join_Ignite_NoSql', 'Join_MarkLogic_Sql', 'Join_MarkLogic_NoSql',
       'Join_Virtuoso_Sql', 'Join_Virtuoso_NoSql'],
      dtype='object')


In [16]:
coluna=[]
for row in df.itertuples():
    #print(col.index().tolist()) 
    #print((col)
    linha=[]
    for col in df.columns:
        if col == "PROJECTS":
            linha.append(getattr(row, col))
            #print(getattr(row, col))
        elif col == "COMMITS":
            linha.append(getattr(row, col))
            #print('Commit:',getattr(row, col))
        elif getattr(row, col) == 1:
            #print(df2[x])
            #print('Database:', col)
            linha.append(col)
    coluna.append(linha)
#print(coluna)

Iteration to generate the categorical data to be used in the sequential patterns algorithm

In [17]:
coluna=[]
projant = 'xxxx'
commitant = 0
linha=[]
contproj = 0
for row in df.itertuples():
    commitant = 0
    for col in df.columns:
        #Verifica se está na primeira coluna do dataframe (PROJECTS) para pegar o nome do projeto
        if col == "PROJECTS":
            projatual = getattr(row, col)
        #Verifica se está na segunda coluna do dataframe (COMMITS) para pegar o número da versão
        elif col == "COMMITS":
            commitatual = getattr(row, col)
        #Entra se o projeto da linha atual for o mesmo da linha anterior ou se for a primeira linha do dataframe, ou seja, o primeiro projeto
        elif projatual == projant or row.Index == 0: # projant == 'xxxx':
            #Entra se há indício do BD e se está na mesma versão do projeto, ou seja, se está percorrendo cada coluna do projeto
            #Então grava na lista o nome da coluna do BD onde encontrou-se o valor == 1
            if (getattr(row, col) == 1) and (commitatual == commitant) :
                linha.append(col)
                commitant = commitatual
            #Entra se há indício do BD e se estão em versões diferentes, para delimitar o fim da versão com "-1"
            elif getattr(row, col) == 1 and (commitatual != commitant):
                #Verifica se é a primeira linha do dataframe ou se a lista está vazia para incluir apenas o nome da coluna na lista
                if row.Index == 0 or len(linha) == 0:
                    linha.append(col)
                else: #Senão grava na lista "-1" + o nome da coluna do BD onde encontrou-se o valor == 1
                    linha.append('-1')
                    linha.append(col)
                    commitant = commitatual
         #Entra se o projeto da linha atual for diferente da linha anterior      
        elif projatual != projant:
            #Entra se há indício do BD, então grava na lista o valor "-1 -2" para delimitar o fim da linha para o projeto anterior
            if getattr(row, col) == 1:# and commitatual == 10:
                linha.append('-1 -2')
                #Inclui a linha do projeto anterior que já tem todas as versões com indícios de BD gravados.
                coluna.append(linha)
                #Cria a nova linha 
                linha=[]
                # Inclui o projeto atual na nova linha
                linha.append(col)
                projant = projatual
                commitant = commitatual
            #Entra se não há indício do BD e se as versões são diferentes
            elif getattr(row, col) == 0 and (commitatual != commitant):
                #Se a lista não está vazia, então grava na lista o valor "-1 -2" para delimitar o fim da linha para o projeto anterior
                if len(linha) != 0:
                    linha.append('-1 -2')
                    #Inclui a linha do projeto anterior que já tem todas as versões com indícios de BD gravados.
                    coluna.append(linha)
                    #Cria a nova linha para o projeto atual
                    linha=[]
                projant = projatual
                commitant = commitatual
            contproj = contproj + 1
    projant = projatual 
#print(len(coluna))
#print(contproj)
#print(coluna)

In [18]:
df2 = pd.DataFrame(data = coluna)
df2

,0,1,2,3,4,5,6,7,8,9,...,220,221,222,223,224,225,226,227,228,229
0,H2,Oracle,PostgreSQL,MySQL,-1,H2,Oracle,IBM_DB2,PostgreSQL,SAPAdaptiveServer,...,None,None,None,None,None,None,None,None,None,None
1,GoogleBigQuery,Snowflake,PostgreSQL,MySQL,-1,GoogleBigQuery,Snowflake,AmazonRedshift,PostgreSQL,MS_SQL_Server,...,None,None,None,None,None,None,None,None,None,None
2,Oracle,Ehcache,-1,Oracle,Ehcache,MySQL,-1,Oracle,PostgreSQL,Ehcache,...,None,None,None,None,None,None,None,None,None,None
3,MySQL,-1,MySQL,-1,MySQL,-1,MySQL,-1,MySQL,-1,...,None,None,None,None,None,None,None,None,None,None
4,MySQL,-1,MySQL,-1,MySQL,-1,MySQL,-1,MySQL,-1,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,MariaDB,-1,SQLite,MySQL,-1,MariaDB,SQLite,PostgreSQL,MySQL,-1,...,None,None,None,None,None,None,None,None,None,None
196,FirebaseRealtime,-1,FirebaseRealtime,-1 -2,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
197,H2,-1,H2,MySQL,-1,H2,Oracle,IBM_DB2,PostgreSQL,MS_SQL_Server,...,None,None,None,None,None,None,None,None,None,None
198,PostgreSQL,MySQL,-1,PostgreSQL,MySQL,-1,PostgreSQL,MySQL,-1,PostgreSQL,...,None,None,None,None,None,None,None,None,None,None


Generating the txt input file in the format required by the SPMF library (PrefixSpamwithStrings algorithm)

In [19]:
HISTORICAL_INPUT_SEQUENCIAL = RESOURCE_DIR + os.sep + 'input_sequencial.txt' #add in util.py
file = open(HISTORICAL_INPUT_SEQUENCIAL, "w")
specialChars = "[',']" # Para retirar estes caracteres que são incluídos pelo array
for values in coluna: # Writes to file
    for specialChar in specialChars:
        values = str(values).replace(specialChar, '')
    file.write(str(values) + '\n')
file.close()

Iteration to generate the categorical data to be used in the sequential patterns algorithm with the negations of the BDs (BD removed/replaced)

In [21]:
####Algoritmo valendo para o arquivo com os 326 projetos, retorna 200 registros, ou seja, 200 repos com indício de BD.
coluna=[]

proj = 0
projant = 'xxxx'
commitant = 0
linha=[]
linhaant = []
contproj = 0
noBD = -2
for row in df.itertuples():
    #print(row.Index) 
    #print((col)
    commitant = 0
    for col in df.columns:
        if col == "PROJECTS":
            #proj = proj + 1
            #linha.append(getattr(row, col))
            #print(getattr(row, col))
            projatual = getattr(row, col)
            #print('atual novo:', projatual, 'anterior:', projant)
            
        elif col == "COMMITS":
            commitatual = getattr(row, col)
            #commitant = commitatual
            #linha.append(getattr(row, col))
            #print('Commit:',getattr(row, col))
        elif projatual == projant or row.Index == 0:# projant == 'xxxx':
            if (commitatual == commitant) :
                if (getattr(row, col) == 1):
                    #print('projs iguais e commits iguais','atual:', projatual,'anterior:', projant, commitatual, commitant, row.Index)
                    #print(df2[x])
                    #print('Database:', col)
                    linha.append(col)
                    linhaant.append(col)
                    #print('entrou commits')
                    commitant = commitatual
                elif getattr(row, col) == 0:
                    #Se for a primeira coluna e a linhaant tinha BD
                    if len(linha) == 0 and linhaant != 0:
                        try:
                            noBD = linhaant.index(col)
                        except:
                            noBD = -1
                        if noBD >= 0: #and row.index == 0:
                            linha.append('No'+col)
                        #print('primeira coluna',noBD, linha, col)
                    elif len(linha) != 0 and linhaant != 0:
                        try:
                            noBD = linha.index(col)
                        except:
                            noBD = -1
                        if noBD >= 0: #and row.index == 0:
                            linha.append('No'+col)
                            #print('demais colunas',noBD, linhaant, col)
                        #print(noBD, linha, col)
            elif (commitatual != commitant):
                if getattr(row, col) == 1:
                    #print('projs iguais ecommits diferentes','atual:', projatual,'anterior:', projant, commitatual, commitant, row.Index )
                    if row.Index == 0 or len(linha) == 0:
                        linha.append(col)
                        linhaant.append(col)
                    else:
                        linha.append('-1')
                        linha.append(col)
                        linhaant.append('-1')
                        linhaant.append(col)
                        commitant = commitatual
        elif projatual != projant:
            if getattr(row, col) == 1:# and commitatual == 10:
                #linha.append('-1 -2 ' + col)
                linha.append('-1 -2')
                #linhaant.append(linha)
                coluna.append(linha)
                linha=[]
                linhaant=[]
                linha.append(col)
                linhaant.append(col)
                #contproj = contproj + 1
                #print('projs diferentes:','atual:', projatual,'anterior:', projant)
                projant = projatual
                commitant = commitatual
            elif getattr(row, col) == 0 and (commitatual != commitant):
                if len(linha) != 0:
                    linha.append('-1 -2')
                    #linhaant.append(col)
                    coluna.append(linha)
                    linha=[]
                    linhaant=[]
                    #print('entrou proj e commits diferentes', 'atual:', projatual,commitatual,'anterior:', projant,commitant )
                projant = projatual
                commitant = commitatual
                
             #novo   
            elif getattr(row, col) == 0 and (commitatual == commitant):
                if len(linha) != 0:
                    #linha.append('-1 -2')
                    #linhaant.append(col)
                    #coluna.append(linha)
                    #linha=[]
                    linhaant=[]
                    #print('entrou', 'atual:', projatual,commitatual,'anterior:', projant,commitant )
                projant = projatual
                commitant = commitatual

            contproj = contproj + 1
        #projant=projatual
        #commitant = 0
    projant = projatual 
    #print ('atual:', projatual,'anterior:', projant)
    #coluna.append(linha)
    #matriz.append(coluna)
    #linha=[]
#matriz.append(coluna)
print(len(linha))
#print(len(linhaant),linhaant)
#print(len(linha))
print(contproj)
#print(coluna)

22
326


Generating the txt input file with the negations of the DBs in the format required by the SPMF library (PrefixSpamwithStrings algorithm)

In [23]:
HISTORICAL_INPUT_SEQUENCIAL_NEG = RESOURCE_DIR + os.sep + 'input_sequencial_neg.txt' #add in util.py
file = open(HISTORICAL_INPUT_SEQUENCIAL_NEG, "w")
#file = open("input_sequencial_neg.txt", "w")
specialChars = "[',']" # Para retirar estes caracteres que são incluídos pelo array
for values in coluna: # Writes to file
    for specialChar in specialChars:
        values = str(values).replace(specialChar, '')
    file.write(str(values) + '\n')
file.close()